In [ ]:
from numpy import zeros, dot, conj, prod, sqrt, exp, pi, diag, angle, array, argwhere, real, floor, frombuffer, uint8, where, stack, asarray
from numpy.linalg import qr, multi_dot, svd
from numpy.random import uniform, normal, randint
import matplotlib.pyplot as plt
import pandas as pd
import io
import cv2
import pickle
import time

In [ ]:
def codebook_gen(num_ant, bit_codebook):
        """
        Generate DFT codebbok of matrix preocders
        :return: matrix of matrix preocders
        """
        num_precoders = 2**bit_codebook
        codebook = zeros(num_precoders, dtype=object)

        for p in range(0, num_precoders):
            precoder = zeros((num_ant, num_ant), dtype=complex)
            for m in range(0, num_ant):
                for n in range(0, num_ant):
                    w = exp(1j*2*pi*(n/num_ant)*(m + p/num_precoders))
                    precoder[n, m] = (1/sqrt(num_ant))*w

            codebook[p] = precoder

        return codebook

def crop_center(img,cropx,cropy):
    y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

def get_img_from_fig(fig, dpi=180):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi)
    buf.seek(0)
    img_arr = frombuffer(buf.getvalue(), dtype=uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    scaled_image = cv2.resize(img, (100, 100))  
#     print(scaled_image.shape)
    
    cropped_img = crop_center(scaled_image, 70, 70)
    cropped_img = cv2.bitwise_not(cropped_img)
    return cropped_img

In [ ]:
bit_codebook = 2
num_ant = 2

In [ ]:
start = time.process_time()
codebook = codebook_gen(num_ant, bit_codebook)
num_classes = 2**bit_codebook
plot_img_np = list()
for prec in range(num_classes):
    precoder = codebook[prec]
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(precoder.real, precoder.imag, 'o', color='black')
    plt.xlim((-1, 1))
    plt.ylim((-1, 1))
    plt.close(fig)
    plot_img_np.append(get_img_from_fig(fig))

print(time.process_time() - start)

In [ ]:

start = time.process_time()
max_iter = 100000

# df = pd.DataFrame(columns = ['Antennas','Bit Codebook','Obs Precoder', 'Correct PMI'])
precoder_img = list()
tx_PMI = list()
for iter in range(max_iter):
    
    PMI = randint(0, num_classes) # generate random precoder index
    tx_PMI.append(PMI)
    precoder_img.append(plot_img_np[PMI]) # pick precoder from codebook
    

    

precoder_img_data = stack(precoder_img, axis=0)
precoder_labels = array(tx_PMI)
print(time.process_time() - start)      

# num_images = max_iter
# for i in range(num_images):
#     plt.imshow(precoder_img_data[i, :, :], cmap = plt.cm.gray)
#     plt.show()

In [ ]:
import torch 
from torch.utils.data import Dataset, DataLoader, random_split

In [ ]:
class PrecoderDataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
        return x, y
    def __len__(self):
        return len(self.data)

precoder_dataset = PrecoderDataset(precoder_img_data, precoder_labels)
train, val = random_split(precoder_dataset, [int(0.8*max_iter), int(0.2*max_iter)])
train_loader = DataLoader(
    train,
    batch_size=100,
    shuffle=True,
    num_workers=0,
)
# increase batch size
val_loader = DataLoader(
    val,
    batch_size=100,
    shuffle=True,
    num_workers=0,
)

In [ ]:
for batch_idx, (data, target) in enumerate(train_loader):
    print('Train: Batch idx {}, data shape {}, target shape {}'.format(
        batch_idx, data.shape, target.shape))
    
for batch_idx, (data, target) in enumerate(val_loader):
    print('Val: Batch idx {}, data shape {}, target shape {}'.format(
        batch_idx, data.shape, target.shape))

In [ ]:
from torch import nn 
from torch import optim 
from torchsummary import summary

In [ ]:
model = nn.Sequential(
        nn.Linear(70*70, 2000),
        nn.ReLU(),
        nn.Linear(2000, 64),
        nn.ReLU(),
        nn.Linear(64, num_classes),
        nn.Softmax(),
)
optimizer = optim.SGD(model.parameters(), lr=1e-4)
loss = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 500 # Number of times you go through the whole dataset
for epoch in range(num_epochs):
    losses = list()
    for batch in train_loader:
        
        x, y = batch # extracted from the batch 
        
        # x: batch size x 1 channel x height x width 
        batch_size = x.size(0)

        x = x.view(batch_size, -1) # -1 means it will mutiply all the dimensions that are not the batch size
        # no the size of x will be batch_size rows and height*width columns

        # step 1: forward pass 
        prob = model(x) 
        
        output = prob
        target = y
        
        # step 2: compute objective function - measuring distance between the output of the network vs actual answer 
        obj_func = loss(prob, y)
        
        # step 3: clear the gradients 
        model.zero_grad()
        
        # step 4: accumulate partial derivatives of obj_func wrt parameters 
        obj_func.backward()
        
        # step 5: step in the opposite direction of the gradient 
        optimizer.step()

        losses.append(obj_func.item())
        
    print(f'Epoch {epoch + 1}, training loss: {torch.tensor(losses).mean():.2f}')